In [ ]:
import os

# Utility functions for binary and text conversions
def string_to_binary(text):
    """Converts a string to its binary representation."""
    return ''.join(format(ord(char), '08b') for char in text)

def binary_to_string(binary):
    """Converts binary data back to a string, including punctuation."""
    chars = [binary[i:i + 8] for i in range(0, len(binary), 8)]
    output = ""
    for char in chars:
        value = int(char, 2)
        if 0 <= value <= 255:  # Allow full ASCII range
            output += chr(value)
        else:
            output += "?"  # Fallback for unsupported values
    return output


def pad_binary(bits, size):
    """Pads binary data to the specified size."""
    return bits.zfill(size)[:size]  # Pad or truncate to the exact size

def permute(k, arr):
    """Applies a permutation using the specified table."""
    if len(k) < max(arr):
        raise ValueError(f"Input binary string is too short for the permutation table. Expected at least {max(arr)} bits, got {len(k)}.")
    return ''.join(k[i - 1] for i in arr)

def xor(a, b):
    """Performs XOR operation on two binary strings."""
    if len(a) != len(b):
        raise ValueError("Lengths of binary strings do not match for XOR.")
    return ''.join('1' if a[i] != b[i] else '0' for i in range(len(a)))

# Custom S-Box based on the provided table
custom_sbox = [
    [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
    [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
    [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
    [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
]

def apply_sbox(bits):
    """Applies the S-Box transformation."""
    output = ""
    for i in range(0, len(bits), 6):
        block = bits[i:i + 6]
        row = int(block[0] + block[5], 2)
        col = int(block[1:5], 2)
        val = custom_sbox[row][col]
        output += format(val, '04b')
    return output

def encrypt_chunk(chunk, key, vigenere_key):
    """Encrypts a 64-bit chunk."""
    if len(chunk) != 64:
        raise ValueError("Chunk size must be 64 bits.")

    # Initial permutation
    permuted = permute(chunk, [
        58, 50, 42, 34, 26, 18, 10, 2,
        60, 52, 44, 36, 28, 20, 12, 4,
        62, 54, 46, 38, 30, 22, 14, 6,
        64, 56, 48, 40, 32, 24, 16, 8,
        57, 49, 41, 33, 25, 17, 9, 1,
        59, 51, 43, 35, 27, 19, 11, 3,
        61, 53, 45, 37, 29, 21, 13, 5,
        63, 55, 47, 39, 31, 23, 15, 7
    ])

    left, right = permuted[:32], permuted[32:]

    # Convert Vigenère key to binary
    vigenere_key_binary = string_to_binary(vigenere_key)
    vigenere_key_binary = pad_binary(vigenere_key_binary, 32)  # Ensure it's 32 bits long

    for i in range(8):
        expanded_right = permute(right, [
            32, 1, 2, 3, 4, 5, 4, 5,
            6, 7, 8, 9, 8, 9, 10, 11,
            12, 13, 12, 13, 14, 15, 16, 17,
            16, 17, 18, 19, 20, 21, 20, 21,
            22, 23, 24, 25, 24, 25, 26, 27,
            28, 29, 28, 29, 30, 31, 32, 1
        ])

        round_key = key[i * 48:(i + 1) * 48]
        round_key = pad_binary(round_key, len(expanded_right))

        xored = xor(expanded_right, round_key)
        sbox_output = apply_sbox(xored)

        # Use Vigenère cipher in the last rounds
        if i >= 4:
            transformed = xor(sbox_output, vigenere_key_binary)
        else:
            transformed = sbox_output

        # Permutation
        permuted_output = permute(transformed, [
            16, 7, 20, 21, 29, 12, 28, 17,
            1, 15, 23, 26, 5, 18, 31, 10,
            2, 8, 24, 14, 32, 27, 3, 9,
            19, 13, 30, 6, 22, 11, 4, 25
        ])

        left, right = right, xor(left, permuted_output)

    combined = left + right
    cipher_text = permute(combined, [
        40, 8, 48, 16, 56, 24, 64, 32,
        39, 7, 47, 15, 55, 23, 63, 31,
        38, 6, 46, 14, 54, 22, 62, 30,
        37, 5, 45, 13, 53, 21, 61, 29,
        36, 4, 44, 12, 52, 20, 60, 28,
        35, 3, 43, 11, 51, 19, 59, 27,
        34, 2, 42, 10, 50, 18, 58, 26,
        33, 1, 41, 9, 49, 17, 57, 25
    ])
    return cipher_text

def decrypt_chunk(ciphertext, key, vigenere_key):
    """Decrypts a 64-bit ciphertext chunk."""
    if len(ciphertext) != 64:
        raise ValueError("Ciphertext size must be 64 bits.")

    # Initial permutation (reverse)
    permuted = permute(ciphertext, [
        58, 50, 42, 34, 26, 18, 10, 2,
        60, 52, 44, 36, 28, 20, 12, 4,
        62, 54, 46, 38, 30, 22, 14, 6,
        64, 56, 48, 40, 32, 24, 16, 8,
        57, 49, 41, 33, 25, 17, 9, 1,
        59, 51, 43, 35, 27, 19, 11, 3,
        61, 53, 45, 37, 29, 21, 13, 5,
        63, 55, 47, 39, 31, 23, 15, 7
    ])

    left, right = permuted[:32], permuted[32:]

    # Convert Vigenère key to binary
    vigenere_key_binary = string_to_binary(vigenere_key)
    vigenere_key_binary = pad_binary(vigenere_key_binary, 32)  # Ensure it's 32 bits long

    for i in range(7, -1, -1):
        expanded_left = permute(left, [
            32, 1, 2, 3, 4, 5, 4, 5,
            6, 7, 8, 9, 8, 9, 10, 11,
           12, 13, 12, 13, 14, 15, 16, 17,
            16, 17, 18, 19, 20, 21, 20, 21,
            22, 23, 24, 25, 24, 25, 26, 27,
            28, 29, 28, 29, 30, 31, 32, 1
        ])

        round_key = key[i * 48:(i + 1) * 48]
        round_key = pad_binary(round_key, len(expanded_left))

        xored = xor(expanded_left, round_key)
        sbox_output = apply_sbox(xored)

        # Reverse Vigenère cipher in the last rounds
        if i >= 4:
            transformed = xor(sbox_output, vigenere_key_binary)
        else:
            transformed = sbox_output

        # Permutation
        permuted_output = permute(transformed, [
            16, 7, 20, 21, 29, 12, 28, 17,
            1, 15, 23, 26, 5, 18, 31, 10,
            2, 8, 24, 14, 32, 27, 3, 9,
            19, 13, 30, 6, 22, 11, 4, 25
        ])

        left, right = xor(right, permuted_output), left

    combined = left + right
    plaintext = permute(combined, [
        40, 8, 48, 16, 56, 24, 64, 32,
        39, 7, 47, 15, 55, 23, 63, 31,
        38, 6, 46, 14, 54, 22, 62, 30,
        37, 5, 45, 13, 53, 21, 61, 29,
        36, 4, 44, 12, 52, 20, 60, 28,
        35, 3, 43, 11, 51, 19, 59, 27,
        34, 2, 42, 10, 50, 18, 58, 26,
        33, 1, 41, 9, 49, 17, 57, 25
    ])
    return plaintext
def process_text(text, key, vigenere_key, operation):
    """Processes text or binary data in chunks."""
    result = ""
    if operation == "encrypt":
        for i in range(0, len(text), 8):
            chunk = text[i:i + 8]  # 8-character chunk
            chunk_binary = pad_binary(string_to_binary(chunk), 64)
            encrypted_chunk = encrypt_chunk(chunk_binary, key, vigenere_key)
            result += encrypted_chunk
    elif operation == "decrypt":
        for i in range(0, len(text), 64):  # 64-bit chunk
            chunk_binary = text[i:i + 64]
            decrypted_chunk_binary = decrypt_chunk(chunk_binary, key, vigenere_key)
            decrypted_chunk = binary_to_string(decrypted_chunk_binary)
            result += decrypted_chunk
    return result

if __name__ == "__main__":
    while True:
        option = input("Choose operation (encrypt/decrypt): ").strip().lower()
        if option == "encrypt":
            plaintext = input("Enter plaintext (length must be a multiple of 8 characters): ").strip()
            if len(plaintext) % 8 != 0:
                print("Error: Plaintext length must be a multiple of 8 characters.")
                continue
            key = input("Enter 128-bit key (binary): ").strip()
            if len(key) != 128:
                print("Error: Key must be 128 bits.")
                continue
            vigenere_key = input("Enter Vigenère cipher key: ").strip()
            additive_key = input("Enter additive cipher key (integer): ").strip()  # Placeholder
            ciphertext = process_text(plaintext, key, vigenere_key, "encrypt")
            print("Encrypted Binary:", ciphertext)
        elif option == "decrypt":
            ciphertext = input("Enter ciphertext (binary, length must be a multiple of 64 bits): ").strip()
            if len(ciphertext) % 64 != 0:
                print("Error: Ciphertext length must be a multiple of 64 bits.")
                continue
            key = input("Enter 128-bit key (binary): ").strip()
            if len(key) != 128:
                print("Error: Key must be 128 bits.")
                continue
            vigenere_key = input("Enter Vigenère cipher key: ").strip()
            additive_key = input("Enter additive cipher key (integer): ").strip()  # Placeholder
            plaintext = process_text(ciphertext, key, vigenere_key, "decrypt")
            print("Decrypted Plaintext:", plaintext)
        else:
            print("Invalid option.")
        cont = input("Do you want to continue? (yes/no): ").strip().lower()
        if cont != "yes":
            print("Goodbye!")
            break


Choose operation (encrypt/decrypt): encrypt
Enter plaintext (length must be a multiple of 8 characters): Cryptographic algorithms protect sensitive data fr!m unauthorize+ access
Enter 128-bit key (binary): 11110101011100110101001111010110011010110010101101011010111001101010011110101100110101100101011001010110010101100010101100101011
Enter Vigenère cipher key: musfiraisstudent
Enter additive cipher key (integer): 63
Encrypted Binary: 00011011010001001001100010110101001011101111011110011001011011101001101100000101101011100011010011001011010001000111110001001001111101100010100101101110011110111111111001101101000110001100000101111010111001101010010011001111010101100010110110010110100101110101000001001000111011000010100111110011010010011110010000001111100001111111101000111110100010011111111101000011111001010111110101101110000110110001001110111011000111011010110001111100111011100111000011000100110111111000010011010000100001000000110010101100011101110110001001001011111100110100010110010100000